# J/psi measurement

### Code to parse the typical STARlight output

In [1]:
import pandas as pd

def parse_starlight_output(file_path):
    events = []
    vertices = []
    tracks = []
    
    with open(file_path, 'r') as file:
        current_event = None
        current_vertex = None
        
        for line in file:
            if line.startswith("EVENT"):
                if current_event:
                    events.append(current_event)
                event_data = line.split()
                current_event = {
                    "Event Number": int(event_data[1]),
                    "Number of Tracks": int(event_data[2]),
                    "Number of Vertices": int(event_data[3]),
                    "Vertices": [],
                    "Tracks": []
                }
            elif line.startswith("VERTEX"):
                vertex_data = line.split()
                current_vertex = {
                    "x": float(vertex_data[1]),
                    "y": float(vertex_data[2]),
                    "z": float(vertex_data[3]),
                    "t": float(vertex_data[4]),
                    "Vertex Number": int(vertex_data[5]),
                    "Process Number": int(vertex_data[6]),
                    "Parent Track": int(vertex_data[7]),
                    "Number of Daughters": int(vertex_data[8])
                }
                current_event["Vertices"].append(current_vertex)
            elif line.startswith("TRACK"):
                track_data = line.split()
                track = {
                    "GPID": int(track_data[1]),
                    "px": float(track_data[2]),
                    "py": float(track_data[3]),
                    "pz": float(track_data[4]),
                    "Event Number": int(track_data[5]),
                    "Track Number": int(track_data[6]),
                    "Stop Vertex": int(track_data[7]),
                    "PDGPID": int(track_data[8])
                }
                current_event["Tracks"].append(track)
        
        if current_event:
            events.append(current_event)
    
    # Flatten the data into lists of dictionaries for DataFrame conversion
    event_list = []
    vertex_list = []
    track_list = []
    
    for event in events:
        event_list.append({
            "Event Number": event["Event Number"],
            "Number of Tracks": event["Number of Tracks"],
            "Number of Vertices": event["Number of Vertices"]
        })
        
        for vertex in event["Vertices"]:
            vertex["Event Number"] = event["Event Number"]
            vertex_list.append(vertex)
        
        for track in event["Tracks"]:
            track_list.append(track)
    
    # Convert to DataFrames
    event_df = pd.DataFrame(event_list)
    vertex_df = pd.DataFrame(vertex_list)
    track_df = pd.DataFrame(track_list)
    
    return event_df, vertex_df, track_df

# Example usage:
# event_df, vertex_df, track_df = parse_starlight_output('starlight_output.txt')

# Display the dataframes
# print(event_df.head())
# print(vertex_df.head())
# print(track_df.head())


### Code to compute the total Lorentz Vector of tracks belonging to the same event

In [2]:
import pandas as pd
import numpy as np
from scipy.constants import c

# Assuming the parse_starlight_output function from the previous code has been defined

def group_tracks_by_event(track_df):
    grouped_tracks = track_df.groupby('Event Number').apply(lambda x: x.to_dict(orient='records')).to_dict()
    return grouped_tracks

# Function to convert track data into a 4-vector for Lorentz operations
def lorentz_vector(track):
    mass = get_mass_from_pdgid(track['PDGPID'])  # Function to get mass based on PDG ID
    energy = np.sqrt(track['px']**2 + track['py']**2 + track['pz']**2 + mass**2)
    return np.array([energy, track['px'], track['py'], track['pz']])

def get_mass_from_pdgid(pdgid):
    # This function would return the mass based on PDG ID
    # For simplicity, assuming some common particle masses
    pdg_masses = {
        11: 0.000511,  # Electron mass in GeV/c^2
        13: 0.10566,   # Muon mass in GeV/c^2
        211: 0.13957,  # Pion mass in GeV/c^2
        # Add more particles as needed
    }
    return pdg_masses.get(abs(pdgid), 0)  # Default to 0 if mass is unknown

# Example of performing Lorentz vector operations
def perform_operations_on_event(grouped_tracks, event_number):
    event_tracks = grouped_tracks[event_number]
    lorentz_vectors = [lorentz_vector(track) for track in event_tracks]
    
    # Example operation: Summing all Lorentz vectors in the event
    total_vector = np.sum(lorentz_vectors, axis=0)
    return total_vector

# Example usage:
# event_df, vertex_df, track_df = parse_starlight_output('starlight_output.txt')
# grouped_tracks = group_tracks_by_event(track_df)
# total_vector = perform_operations_on_event(grouped_tracks, 1)
# print(total_vector)

# The total_vector will be an array [E_total, px_total, py_total, pz_total]


### Code to run STARlight from inside the Jupyter Notebook

In [3]:
import shutil

def copy_file(source_path, destination_path):
    """
    Copies a file from the source path to the destination path.

    Parameters:
    source_path (str): The path to the file you want to copy.
    destination_path (str): The path where the file should be copied.

    """
    try:
        shutil.copy(source_path, destination_path)
        print(f"File copied successfully from {source_path} to {destination_path}")
    except Exception as e:
        print(f"An error occurred while copying the file: {e}")



In [4]:
import subprocess

def modify_and_run_slight_in(input_file_path, output_file_path, mode, n_events, prod_pid=None, starlight_path='../STARlight-build-macos14.4/starlight', output_prefix="output"):
    """
    Modifies specific parameters in the slight.in file, saves the modified file, and runs the STARlight simulation.

    Parameters:
    input_file_path (str): Path to the original slight.in file.
    output_file_path (str): Path where the modified slight.in file will be saved.
    mode (str): The mode of production, either "J/psi" or "photon+photon".
    n_events (int): The number of events to simulate.
    prod_pid (int, optional): The PROD_PID value, relevant for J/psi production.
    starlight_path (str): Path to the STARlight executable.
    output_prefix (str): Prefix for the output files generated by STARlight.

    """
    with open(input_file_path, 'r') as file:
        lines = file.readlines()

    # Initialize the parameters to be modified
    w_min = w_max = w_n_bins = prod_mode = None

    if mode == "J/psi":
        w_min = 3.09645
        w_max = 3.09738
        w_n_bins = 20
        prod_mode = 2
        if prod_pid is None:
            raise ValueError("prod_pid must be provided for J/psi production.")
    elif mode == "photon+photon":
        w_min = 1.0
        w_max = 6.0
        w_n_bins = 400
        prod_mode = 1
        if prod_pid is None:
            prod_pid = 11  # Default to electron if not specified
    else:
        raise ValueError("Invalid mode. Choose either 'J/psi' or 'photon+photon'.")

    # Modify the relevant lines in the slight.in file
    with open(output_file_path, 'w') as file:
        for line in lines:
            if line.startswith("N_EVENTS"):
                file.write(f"N_EVENTS = {n_events}\n")
            elif line.startswith("PROD_MODE") and prod_mode is not None:
                file.write(f"PROD_MODE = {prod_mode}\n")
            elif line.startswith("PROD_PID") and prod_pid is not None:
                file.write(f"PROD_PID = {prod_pid}\n")
            elif line.startswith("W_MIN") and w_min is not None:
                file.write(f"W_MIN = {w_min}\n")
            elif line.startswith("W_MAX") and w_max is not None:
                file.write(f"W_MAX = {w_max}\n")
            elif line.startswith("W_N_BINS") and w_n_bins is not None:
                file.write(f"W_N_BINS = {w_n_bins}\n")
            else:
                file.write(line)

    print(f"Modified slight.in file saved to {output_file_path}")
    copy_file(output_file_path, 'slight.in')

    # Define the output file paths for the STARlight run
    output_file_path = f"{output_prefix}_output.txt"
    error_file_path = f"{output_prefix}_error.txt"


    # Run STARlight using the prepared slight.in file and redirect output
    try:
        command = f"{starlight_path} < {output_file_path}"
        with open(output_file_path, 'w') as output_file, open(error_file_path, 'w') as error_file:
            result = subprocess.run(command, shell=True, stdout=output_file, stderr=error_file, text=True)

        if result.returncode == 0:
            print(f"STARlight executed successfully! Output saved to {output_file_path}")
        else:
            print(f"Error executing STARlight. Check {error_file_path} for details.")

    except Exception as e:
        print(f"An error occurred while trying to run STARlight: {e}")

In [9]:
# Example usage:
# Modify and run a slight.in file for J/psi production with 200000 events and PROD_PID 443011, saving output to custom files
modify_and_run_slight_in('slight-original.in', 'slight-jpsi.in', mode="J/psi", n_events=4000, prod_pid=443011, output_prefix="Jpsi_run1")


Modified slight.in file saved to slight-jpsi.in
File copied successfully from slight-jpsi.in to slight.in
STARlight executed successfully! Output saved to Jpsi_run1_output.txt


In [7]:
# Modify and run a slight.in file for photon+photon production with 100000 events, saving output to custom files
modify_and_run_slight_in('slight-original.in', 'slight-gg.in', mode="photon+photon", n_events=100000, output_prefix="photon_run1")

Modified slight.in file saved to slight-gg.in
File copied successfully from slight-gg.in to slight.in


KeyboardInterrupt: 